In [48]:
from sklearn.base import BaseEstimator, TransformerMixin

class BaseColumnsSelector(BaseEstimator, TransformerMixin):
    def __init__(self, subset):
        self.subset = subset
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.loc[:, self.subset]
    
baseColumns = ['INCOME', 'SAVINGS', 'DEBT', 'T_CLOTHING_12', 'T_CLOTHING_6', 
       'T_EDUCATION_12', 'T_EDUCATION_6', 'T_ENTERTAINMENT_12',
       'T_ENTERTAINMENT_6', 'T_FINES_12',
       'T_FINES_6', 'T_GAMBLING_12', 'T_GAMBLING_6', 
       'T_GROCERIES_12', 'T_GROCERIES_6', 'T_HEALTH_12', 'T_HEALTH_6',
       'T_HOUSING_12', 'T_HOUSING_6', 'T_TAX_12', 'T_TAX_6', 'T_TRAVEL_12',
       'T_TRAVEL_6', 'R_TRAVEL', 'T_UTILITIES_12', 'T_UTILITIES_6', 
       'T_EXPENDITURE_12', 'T_EXPENDITURE_6', 
       'CAT_GAMBLING', 'CAT_CREDIT_CARD', 'CAT_MORTGAGE',
       'CAT_SAVINGS_ACCOUNT', 'CAT_DEPENDENTS']


groups = ['CLOTHING', 'EDUCATION', 'ENTERTAINMENT', 'FINES', 'GAMBLING', 'GROCERIES', 'HEALTH', 'HOUSING', 'TAX', 'TRAVEL', 'UTILITIES']
no_sense_in_division =['T_EDUCATION_12', 'T_FINES_12', 'T_GAMBLING_12', 'T_HOUSING_12', 'T_TAX_12', 'T_TRAVEL_12']
division_groups = ['CLOTHING',  'ENTERTAINMENT',  'GROCERIES', 'HEALTH',  'UTILITIES']


In [49]:

class OutliersReplacer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for column in X.columns:
            if column == 'SAVINGS':
                X.loc[X[column] > 2500000, column] = 2500000
            elif column == 'DEBT':
                X.loc[X[column] > 4000000, column] = 4000000
            elif column == 'T_CLOTHING_12':
                X.loc[X[column] > 32000, column] = 32000
            elif column == 'T_CLOTHING_6':
                X.loc[X[column] > 25000, column] = 25000
            elif column == 'T_HEALTH_12':
                X.loc[X[column] > 25000, column] = 25000
            elif column == 'T_HEALTH_6':
                X.loc[X[column] > 18000, column] = 18000
            elif column == 'T_TRAVEL_12':
                X.loc[X[column] > 150000, column] = 150000
            elif column == 'T_TRAVEL_6':
                X.loc[X[column] > 110000, column] = 110000
        return X

In [50]:
class RatioColumnsAdder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for idx, row in X.iterrows():
            #DEBT/INCOME
            if row['INCOME'] == 0:
                X.at[idx, 'R_DEBT_INCOME'] = None
            else:
                X.at[idx, 'R_DEBT_INCOME'] = row['DEBT'] / row['INCOME']
            #SAVINGS/INCOME
            if row['INCOME'] == 0:
                X.at[idx, 'R_SAVINGS_INCOME'] = None
            else:
                X.at[idx, 'R_SAVINGS_INCOME'] = row['SAVINGS'] / row['INCOME']
            #DEBT/SAVINGS
            if row['SAVINGS'] == 0:
                X.at[idx, 'R_DEBT_SAVINGS'] = None
            else:
                X.at[idx, 'R_DEBT_SAVINGS'] = row['DEBT'] / row['SAVINGS']
            for group in division_groups:
                if row['T_' + group + '_12'] == 0:
                    X.at[idx, 'R_' + group] = None
                else:
                    X.at[idx, 'R_' + group] = row['T_' + group + '_6'] / row['T_' + group + '_12']
            for group in groups:
                if row['INCOME'] == 0:
                    X.at[idx, 'R_'+group+'_INCOME'] = None
                else:
                    X.at[idx, 'R_'+group+'_INCOME'] = row['T_'+group+'_6'] / row['INCOME']
                    
                if row['SAVINGS'] == 0:
                    X.at[idx, 'R_'+group+'_SAVINGS'] = None
                else:
                    X.at[idx, 'R_'+group+'_SAVINGS'] = row['T_'+group+'_6'] / row['SAVINGS']

                if row['DEBT'] == 0:
                    X.at[idx, 'R_'+group+'_DEBT'] = None
                else:
                    X.at[idx, 'R_'+group+'_DEBT'] = row['T_'+group+'_6'] / row['DEBT']
        return X

In [51]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

class MissingValuesFiller(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        numerical_columns = X.select_dtypes(include=[np.number]).columns
        imputer = KNNImputer(n_neighbors=7)
        X[numerical_columns] = imputer.fit_transform(X[numerical_columns])
        return X

In [52]:
class DropColumns(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
    #     potentialColumnsToDrop = ['T_EDUCATION_12', 'T_FINES_12', 'T_GAMBLING_12', 'T_HOUSING_12', 'T_TAX_12', 'T_TRAVEL_12', 'T_EDUCATION_6','T_ENTERTAINMENT_6','T_GAMBLING_6','T_GROCERIES_6','T_HOUSING_6','T_EXPENDITURE_12', 'T_EXPENDITURE_6','R_GROCERIES_DEBT', 'INCOME', 'T_UTILITIES_6', 'R_EDUCATION_DEBT', 'T_UTILITIES_12', 'R_CLOTHING_DEBT',
    #     'CAT_DEPENDENTS', 'R_ENTERTAINMENT_SAVINGS', 'R_FINES_INCOME',
    #     'R_FINES_SAVINGS', 'R_FINES_DEBT', 'R_GROCERIES_SAVINGS',
    #     'CAT_SAVINGS_ACCOUNT', 'R_HOUSING_INCOME', 'R_TAX_INCOME',
    #     'R_TAX_SAVINGS', 'R_TRAVEL_DEBT', 'R_UTILITIES_DEBT', 'CAT_GAMBLING',
    #     'CAT_DEBT', 'CAT_MORTGAGE', 'SAVINGS', 'R_UTILITIES_SAVINGS', 'R_EDUCATION', 'R_FINES', 'R_GAMBLING', 'R_HOUSING', 'R_GROCERIES_INCOME', 'T_ENTERTAINMENT_12', 'R_ENTERTAINMENT',
    #    'R_TRAVEL_SAVINGS', 'R_GAMBLING_SAVINGS', 'T_CLOTHING_6']
        potentialColumnsToDrop = ['T_EDUCATION_12', 'T_FINES_12', 'T_GAMBLING_12', 'T_HOUSING_12', 'T_TAX_12', 
                'T_TRAVEL_12', 'T_EDUCATION_6', 'T_ENTERTAINMENT_6', 'T_ENTERTAINMENT_12', 'T_GAMBLING_6', 
                'T_GROCERIES_6', 'T_HOUSING_6', 'T_EXPENDITURE_12', 'T_EXPENDITURE_6', 'R_GROCERIES_DEBT', 
                'INCOME', 'T_UTILITIES_6', 'T_FINES_6', 'R_FINES_INCOME', 'R_FINES_DEBT', 'CAT_GAMBLING', 
                'CAT_DEBT', 'CAT_MORTGAGE', 'CAT_SAVINGS_ACCOUNT', 'CAT_DEPENDENTS', 'R_FINES_SAVINGS', 
                'CAT_CREDIT_CARD']
        for column in potentialColumnsToDrop:
            if column in X.columns:
                X.drop(column, axis=1, inplace=True)
        return X

In [53]:
from joblib import load

xgboost = load('../models/xgboost_pipe.joblib')
lr = load('../models/lr_pipe.joblib')
gradient_boosting = load('../models/gradient_boosting_pipe.joblib')
svc=load('../models/svc_pipe.joblib')

In [54]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_all = pd.read_csv('../data/data.csv')

train, test = train_test_split(data_all, test_size=0.2, random_state=42)

X_train = train.drop(['CREDIT_SCORE','DEFAULT'], axis=1)
y_train = train['DEFAULT']

X_test = test.drop(['CREDIT_SCORE','DEFAULT'], axis=1)
y_test = test['DEFAULT']

In [55]:
gradient_boosting.fit(X_train,y_train)
lr.fit(X_train,y_train)
svc.fit(X_train,y_train)

Pipeline(steps=[('selector',
                 BaseColumnsSelector(subset=['INCOME', 'SAVINGS', 'DEBT',
                                             'T_CLOTHING_12', 'T_CLOTHING_6',
                                             'T_EDUCATION_12', 'T_EDUCATION_6',
                                             'T_ENTERTAINMENT_12',
                                             'T_ENTERTAINMENT_6', 'T_FINES_12',
                                             'T_FINES_6', 'T_GAMBLING_12',
                                             'T_GAMBLING_6', 'T_GROCERIES_12',
                                             'T_GROCERIES_6', 'T_HEALTH_12',
                                             'T_HEALTH_6', 'T_HOUSING_12',
                                             'T_HOUSING_6', 'T_TAX_12',
                                             'T_TAX_...
                                             'T_TRAVEL_6', 'R_TRAVEL',
                                             'T_UTILITIES_12', 'T_UTILITIES_6',
                                             'T_EXPENDITURE_12',
                                             'T_EXPENDITURE_6', 'CAT_GAMBLING',
                                             'CAT_CREDIT_CARD', ...])),
                ('outliers_replacer', OutliersReplacer()),
                ('ratio_columns_adder', RatioColumnsAdder()),
                ('missing_values_filler', MissingValuesFiller()),
                ('drop_columns', DropColumns()), ('scaler', StandardScaler()),
                ('classifier', SVC(C=10, kernel='linear'))])

In [56]:

y_pred=gradient_boosting.predict(X_test)

In [57]:
y_pred=xgboost.predict(X_test)

In [58]:
y_pred=lr.predict(X_test)

In [59]:
y_pred=svc.predict(X_test)

In [60]:
from sklearn.metrics import accuracy_score, recall_score, precision_score





accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")

Accuracy: 0.74375
Recall: 0.17391304347826086
Precision: 0.7272727272727273


In [61]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Stack the pipelines
stacked_pipeline = StackingClassifier(
    estimators=[
        ('XGBoost', xgboost),
        ('SVC', svc),
        ('gradient_boosting', gradient_boosting),
        ('LogisticRegression', lr)
    ],
    final_estimator=LogisticRegression(),
    stack_method='auto', 
    n_jobs=-1 
)


In [66]:
stacked_pipeline.fit(X_train, y_train)


StackingClassifier(estimators=[('XGBoost',
                                Pipeline(steps=[('selector',
                                                 BaseColumnsSelector(subset=['INCOME',
                                                                             'SAVINGS',
                                                                             'DEBT',
                                                                             'T_CLOTHING_12',
                                                                             'T_CLOTHING_6',
                                                                             'T_EDUCATION_12',
                                                                             'T_EDUCATION_6',
                                                                             'T_ENTERTAINMENT_12',
                                                                             'T_ENTERTAINMENT_6',
                                                                             'T_FINES_12',
                                                                             'T_FINES_6',
                                                                             'T_GAMBLING_12',
                                                                             'T_GAMBLING_6',
                                                                             'T_GROCERIES_12',
                                                                             'T_GROCERIES_6',
                                                                             'T_HEALTH_12',
                                                                             'T_HEALTH_6',
                                                                             'T_HO...
                                                                             'T_EXPENDITURE_6',
                                                                             'CAT_GAMBLING',
                                                                             'CAT_CREDIT_CARD', ...])),
                                                ('outliers_replacer',
                                                 OutliersReplacer()),
                                                ('ratio_columns_adder',
                                                 RatioColumnsAdder()),
                                                ('drop_columns', DropColumns()),
                                                ('missing_values_filler',
                                                 MissingValuesFiller()),
                                                ('scaler', StandardScaler()),
                                                ('classifier',
                                                 LogisticRegression(C=10,
                                                                    solver='newton-cg'))]))],
                   final_estimator=LogisticRegression(), n_jobs=-1)

In [70]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the stacking classifier
cv_scores = cross_val_score(stacked_pipeline, X_train, y_train, cv=16)

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())


/Users/glebbadejko/anaconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1083: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/Users/glebbadejko/anaconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1083: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/Users/glebbadejko/anaconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1083: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/Users/glebbadejko/anaconda3/lib/python3.11/site-packages/sklearn/utils/extmath.py:1083: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2


Cross-validation scores: [0.775 0.75  0.7   0.725 0.75  0.7   0.725 0.75  0.75  0.825 0.7   0.7
 0.725 0.675 0.775 0.7  ]
Mean cross-validation score: 0.7328125


In [67]:
y_pred=stacked_pipeline.predict(X_test)

In [73]:
# from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report




print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.74375
[[109   5]
 [ 36  10]]
              precision    recall  f1-score   support

           0       0.75      0.96      0.84       114
           1       0.67      0.22      0.33        46

    accuracy                           0.74       160
   macro avg       0.71      0.59      0.58       160
weighted avg       0.73      0.74      0.69       160

